## Import and setup

In [1]:
import numpy as np
import os
from tqdm.notebook import tqdm
import processing_utils

In [2]:
if not os.path.exists("./product_amats"):
    os.makedirs("./product_amats")

## Generate all matrices, and split into files of 100,000 lines each

In [3]:
# takes 5 mins on a single core - un-comment below to run

!python 000_script_demo.py > ./product_amats/output.txt

In [4]:
!split -l 100000 --numeric-suffixes ./product_amats/output.txt ./product_amats/output_split_

## save as int8 numpy arrays to reduce size

In [ ]:
for i in tqdm(range(14)):
    _ = processing_utils.save_npy8(i)


  0%|          | 0/14 [00:00<?, ?it/s]

## remove text files to free up storage

In [ ]:
!rm ./product_amats/output*

## make transformation matrix by subtracting product from each possible starting material

### import starting material matrices

In [ ]:
import SM_amats

if not os.path.exists("./rxn_mats"):
    os.makedirs("./rxn_mats")
    
# uncomment below to check it out:
# SM_amats.ac3_am3

In [ ]:
hybr_types = {"ac3_am3":SM_amats.ac3_am3, 
              "ac2_am3":SM_amats.ac2_am3, 
              "ac3_am2":SM_amats.ac3_am2,
              "ac2_am2":SM_amats.ac2_am2,}

### calculate

In [ ]:
def calc_changes(hybr_type):
    
    for file_index in range(18):
        
        # load relevant npy matrix of amats (3D)
        file_tag = str(file_index).zfill(2)
        amats = np.load(f"./product_amats/pdt_amat_{file_tag}_int8.npy")

        # subtract indicated starting material and save as int8
        
        dmats = amats - np.array(hybr_types[hybr_type])
        dmats = dmats.astype("int8")
        np.save(f"./rxn_mats/dmats_{hybr_type}_{file_tag}",dmats)

In [ ]:
for h_t in tqdm(hybr_types):
    calc_changes(h_t)